### Función para descomprimir los datos del socket

In [65]:
def lzw_decompress(compressed_str):
    compressed = [ord(c) for c in compressed_str]
    dict_size = 256
    dictionary = {i: chr(i) for i in range(dict_size)}

    result = []
    w = chr(compressed.pop(0))
    result.append(w)

    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError(f"Bad compressed k: {k}")
        result.append(entry)

        # Add w+entry[0] to the dictionary
        dictionary[dict_size] = w + entry[0]
        dict_size += 1

        w = entry

    return "".join(result)


### Funcion para sacar el pais con lat y lon

In [66]:
import reverse_geocoder as rg

def obtener_pais(lat, lon):
    try:
        results = rg.search((lat, lon))
        return results[0]['name']
    except:
        return 'Desconocido'


### Funcion para limpiar

In [67]:
from datetime import datetime, timezone
import json
def limpiar_datos(d):
    time_ns = d.get("time")

    try:
        # Convertir a segundos desde nanosegundos
        timestamp_sec = int(time_ns) / 1e9
        dt = datetime.fromtimestamp(timestamp_sec, tz=timezone.utc)
        fecha = dt.strftime("%Y-%m-%d")
        hora = dt.strftime("%H:%M:%S")
    except Exception as e:
        print("⚠️ Error al convertir timestamp:", time_ns, e)
        fecha = "1970-01-01"
        hora = "00:00:00"
    
    pais = obtener_pais(d.get("lat"),d.get("lon"))


    datos_limpios = {
        "time": int(time_ns),
        "lat": d.get("lat"),
        "lon": d.get("lon"),
        "alt": d.get("alt"),
        "pol": d.get("pol"),
        "mds": d.get("mds"),
        "mcg": d.get("mcg"),
        "status": d.get("status"),
        "region": d.get("region"),
        "delay": d.get("delay"),
        "lonc": d.get("lonc"),
        "latc": d.get("latc"),
        "fecha": fecha,
        "hora": hora,
        "pais": pais,
        "estaciones": d.get("sig", [])
    }

    return datos_limpios


### Función para insertar los rayos

In [ ]:
import mysql.connector

def guardar_rayo_mysql(data):
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="rayos"
        )
        
        cursor = conn.cursor()
        
        cursor.execute("""
            INSERT INTO rayos (time, lat, lon, alt, pol, mds, mcg, status, region, delay, lonc, latc, fecha, hora, pais)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            data.get("time"),
            data.get("lat"),
            data.get("lon"),
            data.get("alt"),
            data.get("pol"),
            data.get("mds"),
            data.get("mcg"),
            data.get("status"),
            data.get("region"),
            data.get("delay"),
            data.get("lonc"),
            data.get("latc"),
            data.get("fecha"),
            data.get("hora"),
            data.get("pais"),
        ))
        
        id_rayo = cursor.lastrowid
        
        # Insertar hasta 5 estaciones asociadas a este rayo
        for i, est in enumerate(data.get("estaciones", [])):
            if i >= 5:
                break  # solo guardar 5 estaciones como máximo
            cursor.execute("""
                INSERT INTO estaciones (idRayo, sta, tiempoRelat, lat, lon, alt, status)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, (
                id_rayo,
                est.get("sta"),
                est.get("time"),
                est.get("lat"),
                est.get("lon"),
                est.get("alt"),
                est.get("status")
            ))
        
        conn.commit()
        cursor.close()
        conn.close()
        
    except Exception as e:
        print("❌ Error al guardar en MySQL:", e)


# Lectura del socket y guardado a MYSQL XAMPP


In [69]:
import json
import aiohttp
import asyncio

async def escuchar():
    url = "wss://ws2.blitzortung.org/"
    headers = {
        "Origin": "https://www.blitzortung.org",
        "User-Agent": "Mozilla/5.0",
    }

    session = aiohttp.ClientSession()
    contador_rayos = 0

    try:
        async with session.ws_connect(url, headers=headers) as ws:
            await ws.send_str('{"a":111}')  # handshake inicial

            async for msg in ws:
                if msg.type == aiohttp.WSMsgType.TEXT:
                    try:
                        raw = msg.data
                        texto = lzw_decompress(raw)                     # Funcion para descomprimir los datos
                        data = json.loads(texto)
                        rayo_principal = limpiar_datos(data)            # Funcion para limpiar todos los datos y añadir otros
                        
                        if rayo_principal is not None:
                            guardar_rayo_mysql(rayo_principal)          # Funcion para inserte el rayo en MYSQL
                            contador_rayos += 1
                            print(f"✅ Rayo {contador_rayos} guardado")
                        
                        # contador
                        if contador_rayos >= 10:
                            print("✅ Se han procesado 10 rayos. Finalizando...")
                            break
                    except Exception as e:
                        print("❌ Error al procesar mensaje:", e)
                elif msg.type in [aiohttp.WSMsgType.CLOSED, aiohttp.WSMsgType.ERROR]:
                    print("❌ WebSocket cerrado o con error")
                    break
    finally:
        await session.close()

# Ejecutar el bucle
await escuchar()


[{'sta': 1341, 'time': 923466, 'lat': 60.910423, 'lon': 27.420555, 'alt': 85, 'status': 2}, {'sta': 2014, 'time': 1336644, 'lat': 59.425926, 'lon': 24.767048, 'alt': 38, 'status': 12}, {'sta': 1869, 'time': 1630456, 'lat': 53.684048, 'lon': 27.556917, 'alt': 196, 'status': 12}, {'sta': 1144, 'time': 1730472, 'lat': 55.961281, 'lon': 13.100705, 'alt': 81, 'status': 2}, {'sta': 2626, 'time': 1956872, 'lat': 52.675091, 'lon': 14.879443, 'alt': 25, 'status': 4}, {'sta': 1623, 'time': 2145472, 'lat': 51.272049, 'lon': 16.735165, 'alt': 123, 'status': 4}, {'sta': 1692, 'time': 2180778, 'lat': 50.250011, 'lon': 21.484804, 'alt': 182, 'status': 4}, {'sta': 1684, 'time': 2957643, 'lat': 56.844185, 'lon': 35.839256, 'alt': 132, 'status': 12}, {'sta': 2466, 'time': 3181938, 'lat': 48.084435, 'lon': 17.272106, 'alt': 133, 'status': 12}, {'sta': 820, 'time': 3543780, 'lat': 48.457062, 'lon': 12.361052, 'alt': 447, 'status': 10}, {'sta': 1865, 'time': 3566255, 'lat': 47.278931, 'lon': 15.522322, 'al